In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

In [17]:
# Enroll the data 
csv_file = os.path.join( 'Output', 'city_weather.csv')

city_df = pd.read_csv(csv_file)
city_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,5882953,Aklavik,CA,68.2191,-135.0107,-20.20,69,75,3.44
1,909887,Luangwa,ZM,-15.6167,30.4167,91.69,22,35,5.26
2,4023117,Dolores Hidalgo Cuna de la Independencia Nacional,MX,21.1561,-100.9308,60.87,34,48,3.44
3,2170430,Cootamundra,AU,-34.6500,148.0333,53.94,72,100,18.34
4,2037485,Erenhot,CN,43.6475,111.9767,22.95,57,0,4.21
...,...,...,...,...,...,...,...,...,...
567,638936,Rovaniemi,FI,66.5000,25.7167,-4.59,84,75,3.44
568,1655140,Pakxan,LA,18.3778,103.6601,57.22,66,41,3.85
569,2426370,Umm Hājar,TD,13.2954,19.6966,86.88,18,69,4.97
570,1519385,Sergeevka,KZ,53.8800,67.4158,25.36,90,81,11.90


In [18]:
# Humidity Heatmap
gmaps.configure(api_key=g_key)

locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [19]:
# Create new DataFrame

ideal_df = city_df[(city_df['Temperature'] >= 70) & (city_df['Temperature'] <= 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 0]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
72,3896924,Cañete,CL,-37.8000,-73.4000,75.79,38,0,9.08
285,292968,Abu Dhabi,AE,24.4667,54.3667,74.57,64,0,5.75
335,286621,Salalah,OM,17.0151,54.0924,74.97,51,0,1.90
460,3531368,Celestún,MX,20.8667,-90.4000,79.70,50,0,4.32
547,3868707,Valdivia,CL,-39.8142,-73.2459,71.69,26,0,7.96
558,1067565,Beloha,MG,-25.1667,45.0500,79.18,63,0,8.03


In [20]:
# Hotel Mapping

hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
72,3896924,Cañete,CL,-37.8000,-73.4000,75.79,38,0,9.08,
285,292968,Abu Dhabi,AE,24.4667,54.3667,74.57,64,0,5.75,
335,286621,Salalah,OM,17.0151,54.0924,74.97,51,0,1.90,
460,3531368,Celestún,MX,20.8667,-90.4000,79.70,50,0,4.32,
547,3868707,Valdivia,CL,-39.8142,-73.2459,71.69,26,0,7.96,
558,1067565,Beloha,MG,-25.1667,45.0500,79.18,63,0,8.03,


In [21]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }

        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df  

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
72,3896924,Cañete,CL,-37.8000,-73.4000,75.79,38,0,9.08,NaN
285,292968,Abu Dhabi,AE,24.4667,54.3667,74.57,64,0,5.75,NaN
335,286621,Salalah,OM,17.0151,54.0924,74.97,51,0,1.90,NaN
460,3531368,Celestún,MX,20.8667,-90.4000,79.70,50,0,4.32,NaN
547,3868707,Valdivia,CL,-39.8142,-73.2459,71.69,26,0,7.96,NaN
558,1067565,Beloha,MG,-25.1667,45.0500,79.18,63,0,8.03,NaN


In [22]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig 

# Display Map

Figure(layout=FigureLayout(height='420px'))